### Final Project Requirements/notes: https://docs.google.com/document/d/1mwYbYJHkB7kpx4tNflKh54jN9_oOscw3p4k5fsmn3bc/edit

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [3]:
frauds = pd.read_csv('fraudsJoinedwithmaster.csv')
frauds = frauds.dropna()

In [15]:
df = pd.read_csv('numsbyquarter/2018q1/num.txt', sep='\t',encoding ='latin1')
df.head()

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001683168-18-000002,EntityPublicFloat,dei/2014,NaN,20160630,0,USD,3200000.0,NaN
1,0001562762-17-000231,EntityPublicFloat,dei/2014,NaN,20170331,0,USD,28500000.0,NaN
2,0001599916-18-000002,EntityPublicFloat,dei/2014,NaN,20171231,0,USD,253660.0,NaN
3,0001614106-18-000003,EntityPublicFloat,dei/2014,NaN,20161231,0,USD,18247.0,NaN
4,0001144204-18-000327,EntityPublicFloat,dei/2014,NaN,20170331,0,USD,23466799.0,NaN


In [16]:
df = df.drop(columns=['footnote','coreg'])
df = df.dropna()

In [17]:
testfrauds = frauds['adsh']
dftest = df['adsh']

In [18]:
dftest.head()

0    0001683168-18-000002
1    0001562762-17-000231
2    0001599916-18-000002
3    0001614106-18-000003
4    0001144204-18-000327
Name: adsh, dtype: object

In [19]:
testfrauds.head()

0    0000023082-10-000036
1    0000023082-10-000036
2    0000023082-10-000036
3    0000023082-10-000036
4    0000023082-10-000036
Name: adsh, dtype: object

In [20]:
result = dftest[dftest.isin(testfrauds)]
result

Series([], Name: adsh, dtype: object)

In [21]:
combined = frauds.append(df)

In [22]:
combined.shape

(3040863, 11)

In [23]:
# combined = combined.drop(columns=['comp_name','filing_type','related_qtr'])

In [24]:
combined.fraud = combined.fraud.fillna(0)

In [25]:
combined.isnull().sum()

adsh                 0
comp_name      2979958
ddate                0
filing_type    2979958
fraud                0
qtrs                 0
related_qtr    2979958
tag                  0
uom                  0
value                0
version              0
dtype: int64

In [27]:
combined.to_csv('raw_testing_data.csv')

In [28]:
df = pd.read_csv('raw_testing_data.csv')

In [30]:
df.columns

Index(['Unnamed: 0', 'adsh', 'comp_name', 'ddate', 'filing_type', 'fraud',
       'qtrs', 'related_qtr', 'tag', 'uom', 'value', 'version'],
      dtype='object')

In [126]:
benford_df = df[['adsh','value']]
# benford_df

In [127]:
benford_df['abs_value'] = benford_df.value.abs()

In [128]:
benford_df['string'] = benford_df.abs_value.astype(str)
# int(str(number)[0]

In [129]:
# benford_df = benford_df.set_index('adsh')

In [130]:
benford_df['first_digit'] = benford_df.string.str[0].astype(int)

In [131]:
benford_df.head()

,adsh,value,abs_value,string,first_digit
0,0000023082-10-000036,6.360000e+08,6.360000e+08,636000000.0,6
1,0000023082-10-000036,4.090000e+08,4.090000e+08,409000000.0,4
2,0000023082-10-000036,1.264000e+09,1.264000e+09,1264000000.0,1
3,0000023082-10-000036,1.344000e+09,1.344000e+09,1344000000.0,1
4,0000023082-10-000036,-3.417000e+09,3.417000e+09,3417000000.0,3


In [180]:
benford_df = benford_df[['adsh','first_digit']]

In [181]:
type(benford_df['first_digit'][0])

numpy.int64

In [182]:
import numpy as np

In [197]:
benford_pivot = benford_df.pivot_table(index='adsh', columns='first_digit', values='first_digit', aggfunc=np.count_nonzero)

In [198]:
benford_pivot = benford_pivot.fillna(0)
benford_pivot.head()

first_digit,0,1,2,3,4,5,6,7,8,9
adsh,,,,,,,,,,
0000002178-18-000009,111.0,284.0,162.0,142.0,88.0,66.0,56.0,52.0,56.0,38.0
0000002488-18-000042,109.0,372.0,212.0,150.0,86.0,70.0,68.0,78.0,60.0,66.0
0000002969-18-000014,29.0,220.0,112.0,76.0,40.0,36.0,34.0,30.0,22.0,26.0
0000003499-18-000006,21.0,158.0,116.0,126.0,66.0,62.0,16.0,40.0,56.0,20.0
0000003545-18-000021,22.0,166.0,54.0,50.0,48.0,16.0,16.0,10.0,34.0,6.0


In [199]:
benford_pivot = benford_pivot.drop(benford_pivot.columns[0], axis=1)
benford_pivot['sum_of_first_digits'] = benford_pivot.sum(axis=1)
benford_pivot.head()

first_digit,1,2,3,4,5,6,7,8,9,sum_of_first_digits
adsh,,,,,,,,,,
0000002178-18-000009,284.0,162.0,142.0,88.0,66.0,56.0,52.0,56.0,38.0,944.0
0000002488-18-000042,372.0,212.0,150.0,86.0,70.0,68.0,78.0,60.0,66.0,1162.0
0000002969-18-000014,220.0,112.0,76.0,40.0,36.0,34.0,30.0,22.0,26.0,596.0
0000003499-18-000006,158.0,116.0,126.0,66.0,62.0,16.0,40.0,56.0,20.0,660.0
0000003545-18-000021,166.0,54.0,50.0,48.0,16.0,16.0,10.0,34.0,6.0,400.0


In [200]:
benford_pivot.to_excel('first_digit_counts.xls')

In [191]:
# cols = benford_pivot.columns.tolist()
# cols

[1, 2, 3, 4, 5, 6, 7, 8, 9, 'sum_of_first_digits']

In [192]:
# benford_pivot = benford_pivot.reindex(columns= ['sum_of_first_digits', 1, 2, 3, 4, 5, 6, 7, 8, 9])